# Análise de retornos

## Retornos Logaritimicos (LOG-RETORNO)

### LOG do Preço

$$
p_t = \log \left( P_t \right)
$$

### LOG-RETORNO = diferença dos logaritimos dos preços

$$
r_t = p_t - p_{t-1} = \log\left( \frac{P_t}{P_{t-1}} \right)
$$

In [ ]:
import pandas as pd
import numpy as np
from bcb import currency

In [ ]:
usdbrl = currency.get('USD', start_date='2020-01-01', end_date='2020-01-08')

In [ ]:
usdbrl_ask = usdbrl.loc[:, ('USD', 'ask')]

In [ ]:
P_t_1 = usdbrl_ask.shift(1)

In [ ]:
r_t = np.log(usdbrl_ask).diff()

In [ ]:
df = pd.concat([r_t, P_t_1, usdbrl_ask], axis=1)\
    .set_axis(['r_t', 'P_t-1', 'P_t'], axis=1)

In [ ]:
df

## Preços em função dos LOG-RETORNOS

$$
P_t = P_{t-1} \exp\left(r_t\right)
$$

In [ ]:
df['P_t calc'] = df['P_t-1'] * np.exp(df['r_t'])

In [ ]:
df.head()

## Composição dos LOG-RETORNOS

$$
\begin{eqnarray}
P_t & = & P_{t-1} \exp(r_t) \\
P_{t-1} & = & P_{t-2} \exp(r_{t-1}) \\
P_{t-2} & = & P_{t-3} \exp(r_{t-2}) \\
\end{eqnarray}
$$

##### **Escrevendo $P_t$ em termos de $r_{t}$, $r_{t-1}$, $r_{t-2}$**

$$
\begin{eqnarray}
P_{t-2} & = & P_{t-3} \exp(r_{t-2}) \\
P_{t-1} & = & P_{t-3} \exp(r_{t-2}) \exp(r_{t-1}) \\
P_t & = & P_{t-3} \exp(r_{t-2}) \exp(r_{t-1}) \exp(r_t) \\
\end{eqnarray}
$$

##### **Generalizando**

$$
P_t = P_{t-N} \exp\left(\sum_{i=0}^{N-1} r_{t-i} \right)
$$

In [ ]:
df.loc['2020-01-02', 'P_t'] * np.exp( df['r_t'].sum() )

## Reconstruir os Preços a partir dos Retornos

In [ ]:
P_t_recon = df.loc['2020-01-02', 'P_t'] * np.exp(df['r_t'].cumsum())

In [ ]:
P_t_recon

In [ ]:
P_t_recon['2020-01-02'] = df.loc['2020-01-02', 'P_t']

In [ ]:
P_t_recon

In [ ]:
pd.concat([df['P_t'], P_t_recon], axis=1)\
    .set_axis(['P_t', 'P_t_recon'], axis=1)

## Curva de Rentabilidade de um Ativo

In [ ]:
rentabilidade = 100 * np.exp(df['r_t'].cumsum())

In [ ]:
rentabilidade.iloc[0] = 100

In [ ]:
rentabilidade

## Comparar as Rentabilidades de Diversos Ativos

Para 2020, usar:

- USDBRL
- BOVA11
- SMLL11

In [ ]:
import yfinance as yf

In [ ]:
etfs = yf.download(['BOVA11.SA', 'SMAL11.SA'], start='2020-01-01', end='2020-12-31', progress=False)

In [ ]:
etfs.head()

In [ ]:
etfs = etfs['Adj Close']

In [ ]:
usdbrl = currency.get('USD', start_date='2020-01-01', end_date='2020-12-31')

In [ ]:
usdbrl_ask = usdbrl.loc[:, ('USD', 'ask')]

In [ ]:
series = pd.concat([usdbrl_ask, etfs], axis=1)\
    .set_axis(['USDBRL', 'BOVA11', 'SMAL11'], axis=1)

In [ ]:
series.head()

In [ ]:
r_t = series\
    .apply(lambda x: np.log(x).diff())

In [ ]:
r_t.head()

In [ ]:
rentabilidade = 100 * r_t.apply(lambda x: np.exp(x.cumsum()))

In [ ]:
rentabilidade.iloc[0, :] = 100

In [ ]:
rentabilidade.head()

In [ ]:
rentabilidade.plot(figsize=(15,5), grid=True, title='Rentabilidade acumulada');

## Convertendo LOG-RETORNO em Retornos Simples

\begin{equation}
1 + R_t =  \exp\left( r_t \right)
%(\#eq:arithmetic-log-return-relation)
\end{equation}


In [ ]:
R_t = np.exp(r_t).head() - 1

In [ ]:
R_t.head()

## Resumo

- Definição:
    - Variação Percentual $\longrightarrow$ Retornos Simples
    - Retornos Logaritimicos $\longrightarrow$ LOG-RETORNOS: diferença do logaritimos dos preços
- Reconstruindo Preços a partir dos Retornos:
    - Retornos Simples: $P_t = P_{t-N} \Pi_{i=0}^{N-1} (1 + R_{t-i})$
    - LOG-RETORNOS: $P_t = P_{t-N} \exp\left( \sum_{i=0}^{N-1} r_{t-i} \right)$
- Rentabilidade:
    - Retornos Simples Acumulados: $\Pi_{i=0}^{N-1} (1 + R_{t-i})$
    - LOG-RETORNOS Acumulados: $\exp \left( \sum_{i=0}^{N-1} r_{t-i} \right)$